# """
    THIS SCRIPT PERFORM PHOSCPREDICTION FROM IMAGE CROPS
    
    1. THE IMAGE CROPS CAN BE FROM A YOLOV3 PREDICTION SCRIPT 
    2. THE IMAGE CROPS CAN BE FROM A ORIGINAL BB FROM LOCALIZATION DATA
    3. THIS SCRIPT PROJECT BACK THE RESULT ON ORIGINAL IMAGE WHERE WE
    CAN SEE CORRECT AND WRONG PREDICTION.
    4. IMAGE CROPS HAS SPECIFIC NAME FORMAT
    
"""

In [37]:
import tensorflow as tf
tf.__version__

'2.8.0'

"""
This is PHOSCNET model prediction script
"""

In [38]:
from phoscPipeline import * # contains phosc model 
from phos_label_generator import gen_label
from phoc_label_generator import gen_phoc_label
import os

In [39]:
model=build_model()
MODEL="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/new_IAM_16_"
model.load_weights(MODEL+".h5")
#model=tf.keras.models.load_model(MODEL+".h5")
#print(MODEL)

2022-04-13 10:23:33.070957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-13 10:23:33.071254: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aniketag/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-13 10:23:33.071306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aniketag/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-13 10:23:33.071349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dler

In [25]:
"""
    returns a dictionary having word as a key and PHOSC representation as a value 
"""

def get_comb_label(x):
    phos_labels=gen_label(x)
    phoc_labels=gen_phoc_label(x)
    test_labels=dict()
    for x in phos_labels:
        test_labels[x]=np.concatenate((phos_labels[x],phoc_labels[x]),axis=0)
    return test_labels

#train_word_label=get_comb_label(list(set(df_lex['Word'])))

def similarity(x,y):
    return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


"""

    below predict word from  image crop  image  using PHOSCNET 
    
"""

In [2]:
def predict(path2Image,test_word_label):
    
    x=img_to_array(load_img(path2Image))

    x = np.expand_dims(x, axis=0)
    y_pred=model.predict(x)
    y_pred=np.squeeze(np.concatenate((y_pred[0],y_pred[1]),axis=1))
    #print(y_pred)
    mx=0
    for k in test_word_label:
        temp=similarity(y_pred,test_word_label[k])
        if temp>mx:
            mx=temp
            op=k
            
    return op,mx


In [17]:
"""
    gather all words corresponding to specific single image

"""

import re,os

#images=['c04-139.png', 'g06-018r.png', 'a01-072x.png', 'h01-010.png', 'c04-050.png', 'm01-032.png', 'g06-042r.png', 'g06-026i.png', 'h07-080.png', 'e04-022.png','g06-047h.png']

images=["m01-049.png","m01-084.png",'m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

orgImage=images[0]

testCropPath="./data/crops/" #"./data/dataset/forms//" #

#testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/"


def getPhoscFromText(testCropPath,orgImage):
    
    """
        input:image directory and image name containing image crops 
        output: dictionary having word as key and PHOSC representation as value
        
        working: input contains paths to dictionary containing all crops of words present in image
        from labels of image in folder, original word labels are extracted and from it PHOSCNET representation 
        is derived
        
    """
    
    regex = re.compile('[^a-zA-Z]')
    allWords=[]


    for indx,imgName in enumerate(os.listdir(testCropPath+orgImage)):

        word=imgName.split("]_")[1]
        word=word.split("_.png")[0]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    #print(len(allWords))
    #allWords
    return test_word_label,allWords
    
    
#test_word_label,allWords=getPhoscFromText(testCropPath,orgImage)
#len(allWords)

In [18]:
"""
    below part takes input single crop folder name and image name as a input
    and perform PHOSC prediction on it, It also plots the prediction on
    original image
"""

import cv2
import os

regex = re.compile('[^a-zA-Z]')

#testImgPath=".//data/datasets/forms//" 
testCropPath="./data/crops//"+orgImage+"//"
savePath="/data//cropPHOSCVisual//"

print(" orgImage:",orgImage)

if os.path.isfile(testImgPath+orgImage):
    print(" File not present!!!")
else:
    print(" File found!!",testImgPath+orgImage)

    image=cv2.imread(testImgPath+orgImage)
    print(" original image name:",orgImage," original image shape:",image.shape)

matchCount=0
missMatch=0

for indx,cropName in enumerate(os.listdir(testCropPath)):
    
    print("")
    print("cropName:",cropName)
    word=cropName.split("]_")[1]
    word=word.split("_.png")[0]
    word= regex.sub('', word)
    
    if len(word)<2: # skip 
        continue
    
    cor=cropName.split("[")[1]
    cor=cor.split("]")[0]
    cor="["+cor+"]"
    cor=eval(cor)
    
    x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
        
    #imgName=imgName.split("_")[0]
    
    predWord,score=predict(testCropPath+cropName)
    
    print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2)
    
    if word==predWord:
        #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
        #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        matchCount+=1
    else:
        
        cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
        cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        missMatch+=1
    

cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
cv2.imwrite(savePath+orgImage,image)

print("match",matchCount)
print("miss match:",missMatch)
print(" Accuracy:",matchCount/(matchCount+missMatch))


 orgImage: m01-049.png
 File not present!!!

cropName: m01-049.png_108_[1639, 1817, 1812, 1912]_himself_.png


TypeError: predict() missing 1 required positional argument: 'test_word_label'

"""

    below code predicts PHOSC and visualizes all the image present in testCrop folder 
    to add more images into it changes are required in file 
    mnist_testYolo.txt which is in yolo prediction script
    
"""

In [38]:
"""
    gather all words
    
"""
import cv2
import os

import re
regex = re.compile('[^a-zA-Z]')

images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']
#images=['m01-084.png']

#orgImage=images[0]

testImgPath=".//data/dataset/forms//" #"/home/aniketag/Documents/phd/yolov5/data/datasets/forms/m01-084.png"
#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 

testCropPath="./data//crops//"

images=os.listdir(testCropPath)

savePath=".//data/cropPHOSCVisual//"

print(" images:",images[0] in os.listdir(testImgPath))

    
for orgImage in images:
    
    matchCount=0
    missMatch=0
    testCropPath="./data//crops//"
    
    if not os.path.isdir(os.path.join(testCropPath,orgImage)):
        continue
    
    test_word_label,allWords=getPhoscFromText(testCropPath,orgImage)

    testCropPath=testCropPath+orgImage+"//"

    if os.path.isfile(savePath+orgImage+"_singleDict_.png"):
        continue
    
    print(" imageName:",orgImage," No of unique words:",len(allWords)," is file:",os.path.isfile(testImgPath+orgImage))

    image=cv2.imread(testImgPath+orgImage)
    print(" original image shape:",image.shape)

    df=pd.DataFrame(index=range(200),columns=["image_name","crop_name","original_word","predicted_word"])   

    
    for indx,cropImgName in enumerate(os.listdir(testCropPath)):

        #print(" cropImgName:",cropImgName)
                    
        #temp=pd.DataFrame(columns=["image_name","crop_name","original_word","predicted_word"]

        word=cropImgName.split("]_")[1]
        word=word.split("_.png")[0]

        cor=cropImgName.split("[")[1]
        cor=cor.split("]")[0]
        cor="["+cor+"]"
        cor=eval(cor)

        x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])

        #imgName=imgName.split("_")[0]

        predWord,score=predict(testCropPath+cropImgName,test_word_label)

        print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
        print("")
                
        df.loc[indx,"image_name"]=orgImage
        df.loc[indx,"crop_name"]=cropImgName
        df.loc[indx,"original_word"]=word
        df.loc[indx,"predicted_word"]=predWord        

        if word==predWord:
            #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
            #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            matchCount+=1
        else:

            cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
            cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            missMatch+=1


    print("match",matchCount)
    print("miss match:",missMatch)
    print(" Accuracy:",matchCount/(matchCount+missMatch))

    cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
    cv2.imwrite(savePath+orgImage+"_singleDict_.png",image)
    df.to_csv(savePath+orgImage+"_singleDict_.csv")
    
    #input("check!!!")
    

 images: False
 imageName: m01-131.png  No of unique words: 46  is file: True
 original image shape: (3542, 2479, 3)
 word: ,   finally  score: 705.9713209238249  coordinate: 498 1555 519 1588



/tmp/ipykernel_267330/1231027792.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


 word: thought   thought  score: 958.4195036051397  coordinate: 1544 823 1766 958

 word: having   down  score: 973.8037790114391  coordinate: 1316 971 1469 1043

 word: paced   paced  score: 928.3140952481344  coordinate: 510 1117 676 1199

 word: and   and  score: 991.8131587397843  coordinate: 1922 833 2036 892

 word: finally   unbidden  score: 838.0491023813452  coordinate: 669 833 1002 942

 word: action   action  score: 930.0415843438552  coordinate: 1898 708 2089 780

 word: ,   finally  score: 656.5688983519682  coordinate: 1818 1031 1827 1070

 word: down   down  score: 976.0148757112704  coordinate: 355 1509 491 1568

 word: completely   completely  score: 919.4637915170548  coordinate: 1404 1106 1679 1205

 word: the   the  score: 864.920589619335  coordinate: 550 850 653 909

 word: upon   upon  score: 879.8191647070341  coordinate: 635 1003 789 1087

 word: sat   spectators  score: 777.3493540039808  coordinate: 1200 990 1291 1055

 word: Dan   out  score: 807.55228307005

 word: your   agricultural  score: 793.3104692152868  coordinate: 342 1990 542 2078

 word: his   his  score: 769.7757494628181  coordinate: 1076 1629 1171 1685

 word: how   how  score: 929.9908580609206  coordinate: 657 702 812 784

 word: it   it  score: 843.8228109261015  coordinate: 1195 725 1268 777

 word: the   the  score: 875.6151238842915  coordinate: 1154 1254 1274 1326

 word: the   the  score: 981.554253800103  coordinate: 373 886 461 965

 word: area   agricultural  score: 807.2244790626338  coordinate: 854 1261 1072 1343

 word: mouth-organ   would  score: 697.7142398733192  coordinate: 640 1966 903 2047

 word: few   agricultural  score: 819.4248231213347  coordinate: 1894 1404 2089 1530

 word: Lester   craterlet  score: 816.4924956475902  coordinate: 761 1794 1026 1861

 word: craterlet   craterlet  score: 896.496007829767  coordinate: 374 1069 743 1150

 word: visited   visited  score: 904.1000179053071  coordinate: 1679 905 2022 973

 word: .   is  score: 755.170294

 word: ones   ones  score: 899.4522649386836  coordinate: 850 1284 1004 1334

 word: bills   bills  score: 915.9361862028414  coordinate: 1502 1439 1621 1494

 word: love   love  score: 914.8309629448299  coordinate: 1505 1091 1648 1157

 word: had   had  score: 990.5463658762113  coordinate: 338 1448 479 1507

 word: again   again  score: 924.8097924433993  coordinate: 739 734 939 833

 word: metaphysics   parades  score: 835.1714592429844  coordinate: 1143 722 1588 809

 word: of   parades  score: 897.2983508239447  coordinate: 1771 1095 2055 1170

 word: of   off  score: 874.7645799166527  coordinate: 1701 1092 1790 1171

 word: an   an  score: 778.4544834977868  coordinate: 778 1112 884 1158

 word: physical   physical  score: 825.3703290147965  coordinate: 1626 906 1934 992

 word: best   best  score: 904.74790055994  coordinate: 862 1626 999 1687

 word: naturally   naturally  score: 900.0947716850083  coordinate: 1873 1452 2208 1529

 word: once   once  score: 973.4815289647094 

 word: at   at  score: 807.3306514120818  coordinate: 394 2162 493 2236

 word: Finally   Finally  score: 905.327088275484  coordinate: 457 1790 667 1941

 word: vague   awaited  score: 790.2202461382637  coordinate: 1790 1315 2052 1445

 word: obviously   surroundings  score: 864.5007274480256  coordinate: 1138 1810 1470 1964

 word: points   the  score: 898.3517391569288  coordinate: 1197 2679 1336 2758

 word: Steve   worldwide  score: 773.556923354767  coordinate: 901 1986 1143 2062

 word: perplexity   and  score: 997.2058964147463  coordinate: 357 1603 552 1696

 word: perplexity   Finally  score: 885.1670371956549  coordinate: 527 1444 869 1613

 word: perplexity   attitude  score: 854.8598800231936  coordinate: 369 1444 894 1599

 word: the   the  score: 969.7688494406208  coordinate: 688 2509 819 2595

 word: all   all  score: 904.3651184999718  coordinate: 1877 2530 1985 2605

 word: world-wide   attitude  score: 846.3403422767319  coordinate: 1002 2335 1341 2422

 word: .   

 word: my   necessity  score: 692.3826061013053  coordinate: 1458 2005 1600 2062

 word: only   matted  score: 834.0590404089869  coordinate: 1925 1778 2124 1888

 word: ,   saying  score: 639.9016352574344  coordinate: 2074 776 2095 813

 word: an   gazed  score: 771.026607206779  coordinate: 631 1637 745 1670

 word: so   so  score: 826.5217667998676  coordinate: 1132 1810 1221 1857

 word: an   compels  score: 720.5922362999535  coordinate: 1722 1656 1834 1688

 word: hair   hesitation  score: 768.3053220224239  coordinate: 1231 893 1417 952

 word: unusual   intervention  score: 677.89123534589  coordinate: 1648 1795 1674 1851

 word: black   black  score: 934.0098261840319  coordinate: 593 899 800 972

 word: creatures   creatures  score: 886.181354734411  coordinate: 1206 702 1567 771

 word: in   in  score: 768.4844495812453  coordinate: 831 1442 899 1495

match 46
miss match: 43
 Accuracy: 0.5168539325842697


IndexError: list index out of range

In [62]:
print(allWords)
len(allWords)
#images

['', 'light', 'loses', 'lifetempo', 'he', 'abroad', 'was', 'at', 'apartment', 'to', 'intersection', 'pedestrian', 'precaution', 'no', 'an', 'in', 'as', 'change', 'its', 'Listening', 'stood', 'garaged', 'other', 'listening', 'the', 'At', 'beat', 'there', 'nor', 'For', 'his', 'vain', 'and', 'night', 'for', 'wait', 'earth', 'superfluous', 'heart', 'path', 'car']


41

"""

                    BATCH VISUALIZATION

    BELOW PART CAN BE USED TO VISUALIZE THE PERFORMANCE OF ORIGINAL CROPS,
    REQUIRED CROPS ARE TAKEN FROM PHOSCNET TRAINING DATA.
    THE CROP NAMES DOES NOT INCLUDE WORDS PRESENT INSIDE IT OR COORDINATE TO HANDLE IT
    CSV data_444_april.csv IS USED WHICH HAS REQUIRED INFORMATION

"""

"""

    1. Select test image_name from data_444*.csv and fetch all records related to that.
    
    2. From fetched records get BB and text of location IE create a new dataframe which has BB and corresponding text
    * there is difference between image name and crop name
    
    3. From PHOSCNET test data separate all crops corresponding to above recods and create a separate folder for it,
    here we have only Image name from 1,2 but not corresponding crop name so we need to derive
    it with string matching.
    
    Get the PHOSC vector space for all word labels present in speecific image
    
    4. Pass those crop to PHOSCNET get prediction and visualize it..
    
"""

In [13]:
import pandas as pd
import os
import cv2
import re

"""
    allPaths
"""

csvFile="./data/dataset/data_12_april.csv"
phoscTestImgPath="./data//phoscTest//"
allPhoscTestCrops="./data/IAM_Data/IAM_test/"
PhoscOrgTestCrops="./crop2//"

In [7]:

df1=pd.read_csv(csvFile)
print(df1.shape)

phoscTestImg=os.listdir(phoscTestImgPath)
print(" total test images:",len(phoscTestImg))
df1.head(5)

(115187, 14)
 total test images: 232


,image_name,class,width,height,org_x1,org_y1,org_x2,org_y2,text,cropName,x,y,w,h
0,g06-031n.png,1,2479,3542,435,678,450,697,',g06-031n-00-01,0.18,0.19,0.01,0.01
1,g06-031n.png,1,2479,3542,425,683,663,777,What,g06-031n-00-02,0.22,0.21,0.10,0.03
2,g06-031n.png,1,2479,3542,640,726,696,779,a,g06-031n-00-03,0.27,0.21,0.02,0.01
3,g06-031n.png,1,2479,3542,727,685,1128,820,frightful,g06-031n-00-04,0.37,0.21,0.16,0.04
4,g06-031n.png,1,2479,3542,1129,692,1365,779,event,g06-031n-00-05,0.50,0.21,0.10,0.02


In [30]:
"""
1,2.
    input: image name for which BB,Text records are needed
    output: dataframe with relevant information
    
"""

def fetchRecord(df,imageName):
    
    temp=df[df["image_name"]==imageName]
    #print(" No of records:",temp.shape)
    return temp


"""
3.
    crops corresponding to specific image file
    a. get list of all crops used for PHOSCNET testing
    b. filter crops necessary for specific image file
    
"""
from shutil import copy2

def filterCrop(allCropList,imageName):
    
    imgCropList=[]
    
    imageName=imageName.split(".png")[0]
    
    cropPhoscTestData="./data/cropPhoscTestData//"
    basePhoscData="./data//IAM_Data/IAM_test//"
    
    for cropName in allCropList:
        
        if imageName in cropName:
            
            imgCropList.append(cropName)

    print(" TOTAL CROPS CORRESPONDING TO IMAGE "+imageName+" ARE:",len(imgCropList))
    
    
    try:
        os.mkdir(cropPhoscTestData+imageName)
    except Exception as e:
        pass
    
    for filterCropName in imgCropList:
        copy2(basePhoscData+filterCropName,cropPhoscTestData+imageName)
        
        
    print(" transferred crop:",len(cropPhoscTestData+imageName)) 
        
    return imgCropList

#imageName="m01-125.png"
#allCropList=os.listdir(allPhoscTestCrops)
#mgCropList=filterCrop(allCropList,imageName)
#imgCropList

def getPhoscFromDF(df):
    
    """
        input: Dataframe containing all image information like text,coordinate etc.
        output: dictionary having word as key and PHOSC representation as value
        
        working: similar to getPhoscFromText()
        
    """
    
    regex = re.compile('[^a-zA-Z]')
    allWords=[]


    for indx,row in df.iterrows():

        word=row["text"]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    print(len(allWords))
    #allWords
    return test_word_label,allWords

#imageName="m01-084.png"
#temp=fetchRecord(df1,imageName)
#temp.columns
#test_word_label=getPhoscFromDF(temp)
#temp.head(5)

"""
    save crops from original coordinates
    As there is no way to map PHOSCNET test data crops to original image 
    
    below function extracts crops from original image, 
    The crops are saved in following format ImgName.png_IndexNumber_[x1,y1, x2, y2]_textString_.png
    
    The crop coordinate are from original dataset It is not predicted by YOLOV3
    
"""

def PhoscOrgCrops(df,imgName):
    
    image=cv2.imread(phoscTestImgPath+imgName)
    
    #print(image.shape)
    
    regex = re.compile('[^a-zA-Z]')
    
    try:
        os.mkdir(PhoscOrgTestCrops+imgName)
    
    except Exception as e:
        pass
    
    for indx,row in df.iterrows():
        #print(row)
        word=row["text"]
        word= regex.sub('', word)
        
        if len(word)<2:
            continue

        x1,y1,x2,y2=row["org_x1"],row["org_y1"],row["org_x2"],row["org_y2"]
        
        tempName=imgName+"_"+str(indx)+"_"+str([x1,y1,x2,y2])+"_"+word+".png"
        
        cropImage=image[y1:y2,x1:x2]
        #print(cropImage.shape)

        cv2.imwrite(PhoscOrgTestCrops+imgName+"//"+tempName,cropImage)
        


#PhoscOrgCrops(temp,imageName)



In [109]:
"""
    gather all words
"""
import cv2
import os
import sys
import re
regex = re.compile('[^a-zA-Z]')

images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

PhoscOrgTestCrops="./data//crop3//"

images=os.listdir(phoscTestImgPath)

savePath="./data//cropPHOSCVisual//"
testImgPath=".//data/dataset/forms//" 
print(" images:",images[0] in os.listdir(testImgPath))

allCropList=os.listdir(allPhoscTestCrops)


for imgNumber,orgImage in enumerate(images):
    
    try:
        print("")
        
        """
        if os.path.isfile(savePath+orgImage+"_originalCrops_.png"):
            continue
        """ 
            
        if os.path.isfile(savePath+orgImage+"_originalTest_.png"):
            continue

        matchCount=0
        missMatch=0
        PhoscOrgTestCrops="./data//crop3//"

        temp=fetchRecord(df1,orgImage) # get relevant records

        #PhoscOrgCrops(temp,orgImage) # create crops
        
        filterCrop(allCropList,orgImage)
        
        #test_word_label,allWords=getPhoscFromText(PhoscOrgTestCrops,orgImage) # get attribute space

        test_word_label,allWords=getPhoscFromDF(temp)
        
        if os.path.isdir(PhoscOrgTestCrops+orgImage+"//"):
            PhoscOrgTestCrops=PhoscOrgTestCrops+orgImage+"//"
        else:
            PhoscOrgTestCrops=PhoscOrgTestCrops+orgImage[:-4]+"//"            

        #print(" imageName:",orgImage," No of unique words:",len(allWords)," is file:",os.path.isfile(testImgPath+orgImage))

        print(" imageName:",orgImage," No of unique words:",len(test_word_label.keys())," is file:",os.path.isfile(phoscTestImgPath+orgImage))


        image=cv2.imread(phoscTestImgPath+orgImage)
        #print(" original image shape:",image.shape)
    
        
        df=pd.DataFrame(index=range(200),columns=["image_name","crop_name","original_word","predicted_word"])   

        for indx,cropImgName in enumerate(os.listdir(PhoscOrgTestCrops)):

            #print(" cropImgName:",cropImgName)

            #temp=pd.DataFrame(columns=["image_name","crop_name","original_word","predicted_word"]
            
            
            try:
                word=cropImgName.split("]_")[1]
                word=word.split(".png")[0]

                cor=cropImgName.split("[")[1]
                cor=cor.split("]")[0]
                cor="["+cor+"]"
                cor=eval(cor)
                
            except Exception as e:
                tempCropName=cropImgName[:-4]
                tempRow=df1[df1["cropName"]==tempCropName]
                #print("")
                #print(tempRow)
                #print("")

                word=tempRow.text.values[0]
                cor=[tempRow.org_x1.values,tempRow.org_y1.values,tempRow.org_x2.values,tempRow.org_y2.values]
                #input(" check row")
                
                #print(" word=",word)
                #print(" cor=",cor)
                
                
            x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
            
            #imgName=imgName.split("_")[0]

            predWord,score=predict(PhoscOrgTestCrops+cropImgName,test_word_label)
            
            print(" word:",word," predWord:",predWord," score:",score," coordinate:",x1,y1,x2,y2," cropImgName:",cropImgName)
            predWord=predWord[:-3]
            
            try:
                df.loc[indx,"image_name"]=orgImage
                df.loc[indx,"crop_name"]=cropImgName
                df.loc[indx,"original_word"]=word
                df.loc[indx,"predicted_word"]=predWord        
            except Exception as e:
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print(" line number:", exc_tb.tb_lineno, " e:",e)
            
            if word==predWord:
                cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
                #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                matchCount+=1
            else:

                cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
                cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                missMatch+=1


        print("match",matchCount)
        print("miss match:",missMatch)
        print(" Accuracy:",matchCount/(matchCount+missMatch))

        cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
        cv2.imwrite(savePath+orgImage+"_originalTest_.png",image)
        #df.to_csv(savePath+orgImage+"_originalTest_.csv")
            
    except Exception as e1:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(" line number:", exc_tb.tb_lineno, " e:",e1)
        
    #input("check!!!")
    

 images: True

 TOTAL CROPS CORRESPONDING TO IMAGE n04-213 ARE: 57
 transferred crop: 33
60
 imageName: n04-213.png  No of unique words: 60  is file: True
 word: that  predWord: it  score: 998.4719046136418  coordinate: 970 1449 1072 1514  cropImgName: n04-213-03-05.png


/tmp/ipykernel_21847/1231027792.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


 word: but  predWord: she  score: 998.3280429307597  coordinate: 360 1448 461 1516  cropImgName: n04-213-03-01.png
 word: manner  predWord: remained  score: 971.4658412861428  coordinate: 1539 1303 1794 1340  cropImgName: n04-213-02-06.png
 word: There  predWord: had  score: 999.6153682909008  coordinate: 369 905 527 989  cropImgName: n04-213-00-01.png
 word: deal  predWord: of  score: 999.4797456153508  coordinate: 1760 1100 1889 1157  cropImgName: n04-213-01-10.png
 word: common  predWord: lot  score: 996.7164258464633  coordinate: 719 1660 990 1698  cropImgName: n04-213-04-04.png
 word: been  predWord: many  score: 995.9874107895577  coordinate: 749 915 879 983  cropImgName: n04-213-00-03.png
 word: had  predWord: been  score: 998.9656231154648  coordinate: 602 911 701 980  cropImgName: n04-213-00-02.png
 word: His  predWord: normal  score: 992.5901994453496  coordinate: 381 1258 446 1325  cropImgName: n04-213-02-01.png
 word: happenings  predWord: are  score: 998.731152288206  coor

 word: shot  predWord: past  score: 889.4619885090859  coordinate: 608 1895 841 1964  cropImgName: m04-113-06-03.png
 word: the  predWord: bus  score: 962.3946065895522  coordinate: 992 2424 1147 2498  cropImgName: m04-113-09-04.png
 word: dogged  predWord: towards  score: 981.7138973787506  coordinate: 982 1524 1328 1642  cropImgName: m04-113-04-03.png
 word: been  predWord: mistaken  score: 941.9005163391508  coordinate: 1702 2241 1920 2314  cropImgName: m04-113-08-06.png
 word: a  predWord: sign  score: 886.5411902612551  coordinate: 1748 1022 1800 1058  cropImgName: m04-113-01-07.png
 word: an  predWord: accident  score: 967.2341406132897  coordinate: 1447 2102 1582 2141  cropImgName: m04-113-07-07.png
 word: bus  predWord: safe  score: 901.6966753505089  coordinate: 1171 2419 1336 2499  cropImgName: m04-113-09-05.png
 word: ,  predWord: and  score: 999.7287914755243  coordinate: 749 2653 771 2689  cropImgName: m04-113-10-02.png
 word: seen  predWord: no  score: 994.6260242145144  

 word: then  predWord: laid  score: 973.9671164585601  coordinate: 1999 1245 2185 1331  cropImgName: p02-022-04-00.png
 word: the  predWord: kisses  score: 909.444062600347  coordinate: 2041 1958 2175 2056  cropImgName: p02-022-08-00.png
 word: over  predWord: the  score: 871.8200210591137  coordinate: 1336 1264 1511 1326  cropImgName: p02-022-03-05.png
 word: hand  predWord: on  score: 795.0143670914872  coordinate: 704 1425 909 1504  cropImgName: p02-022-04-03.png
 word: kisses  predWord: and  score: 775.9334573345345  coordinate: 324 2129 614 2215  cropImgName: p02-022-08-01.png
 word: on  predWord: was  score: 932.7948761065501  coordinate: 956 1443 1048 1507  cropImgName: p02-022-04-04.png
 word: And  predWord: then  score: 989.4522849946728  coordinate: 385 706 520 787  cropImgName: p02-022-00-02.png
 word: seen  predWord: full  score: 808.2010931639016  coordinate: 1115 2158 1323 2224  cropImgName: p02-022-08-04.png
 word: Gay  predWord: smiled  score: 912.443199246103  coordina

...

 word: his  predWord: hands  score: 901.7795145997504  coordinate: 1517 1445 1621 1524  cropImgName: m04-019-03-05.png
 word: last  predWord: embrace  score: 799.4151476391967  coordinate: 1396 1610 1542 1691  cropImgName: m04-019-04-05.png
 word: me  predWord: it  score: 995.8275747319236  coordinate: 1408 1307 1506 1339  cropImgName: m04-019-02-10.png
 word: miles  predWord: away  score: 779.6155112372707  coordinate: 1638 910 1827 985  cropImgName: m04-019-00-08.png
 word: a  predWord: last  score: 996.9660096223901  coordinate: 1312 1657 1347 1687  cropImgName: m04-019-04-04.png
 word: it  predWord: on  score: 825.5479389285462  coordinate: 1771 1961 1853 2049  cropImgName: m04-019-06-06.png
 word: be  predWord: remembering  score: 855.499257327487  coordinate: 785 1802 836 1875  cropImgName: m04-019-05-03.png
 word: are  predWord: lording  score: 968.1031997760371  coordinate: 1167 2015 1273 2050  cropImgName: m04-019-06-04.png
 word: only  predWord: going  score: 997.716159008936

 word: far  predWord: too  score: 997.9870410150961  coordinate: 1995 898 2100 1002  cropImgName: n04-015-02-00.png
match 0
miss match: 44
 Accuracy: 0.0

 TOTAL CROPS CORRESPONDING TO IMAGE p01-168 ARE: 43
 transferred crop: 33
55
 imageName: p01-168.png  No of unique words: 55  is file: True
 word: to  predWord: make  score: 986.456600676582  coordinate: 1251 758 1334 818  cropImgName: p01-168-00-05.png
 word: described  predWord: afterwards  score: 963.3685748558789  coordinate: 1258 1662 1544 1740  cropImgName: p01-168-05-07.png
 word: make  predWord: of  score: 999.3100879919857  coordinate: 1388 763 1570 815  cropImgName: p01-168-00-06.png
 word: Perhaps  predWord: he  score: 998.6446246320323  coordinate: 1597 931 1862 1024  cropImgName: p01-168-01-07.png
 word: didn't  predWord: know  score: 995.9312645860988  coordinate: 507 743 735 812  cropImgName: p01-168-00-02.png
 word: ,  predWord: hunted  score: 975.4837637376918  coordinate: 2070 1387 2095 1415  cropImgName: p01-168-04

 word: through  predWord: empty  score: 825.1966766299952  coordinate: 423 1125 653 1216  cropImgName: m04-231-02-01.png
 word: Do  predWord: as  score: 984.0795955799609  coordinate: 418 771 492 836  cropImgName: m04-231-00-02.png
 word: the  predWord: driver  score: 928.4951484984524  coordinate: 1163 937 1239 1005  cropImgName: m04-231-01-05.png
 word: of  predWord: food  score: 998.17340845487  coordinate: 1454 1474 1518 1569  cropImgName: m04-231-04-06.png
match 0
miss match: 29
 Accuracy: 0.0

 TOTAL CROPS CORRESPONDING TO IMAGE n06-156 ARE: 46
 transferred crop: 33
48
 imageName: n06-156.png  No of unique words: 48  is file: True
 word: followed  predWord: in  score: 997.6033775135573  coordinate: 634 1842 932 1898  cropImgName: n06-156-06-02.png
 word: miners  predWord: who  score: 998.460659142056  coordinate: 818 960 1021 995  cropImgName: n06-156-01-04.png
 word: couple  predWord: of  score: 999.2233416564779  coordinate: 915 2024 1119 2077  cropImgName: n06-156-07-04.png
 w

 word: seemed  predWord: to  score: 999.3949189316296  coordinate: 1052 1465 1410 1553  cropImgName: m04-107-04-05.png
 word: half  predWord: an  score: 998.2015045211983  coordinate: 718 1820 923 1953  cropImgName: m04-107-06-02.png


KeyboardInterrupt: 

In [22]:
len(os.listdir(phoscTestImgPath))

232

In [101]:
allPhoscTestCrops="./data/IAM_Data/IAM_test/"
len(os.listdir(allPhoscTestCrops))

9659

In [91]:
cropName="n04-213-03-05"
df1.shape,df1.columns

d=df1[df1['cropName']==cropName]
#print(d.reset_index(drop=True))

d.text.values[0]
type(d)
#d

#df1.reset_index(drop=True,inplace=True)
#df1.head(5)

pandas.core.frame.DataFrame

In [84]:
df1=pd.read_csv("/home/aniketag/Documents/phd/yolov5/data/datasets/data_444_april.csv")
df1.shape

temp=df1[df1["image_name"]=="m01-084.png"]
temp.shape
temp.head(10)

,image_name,class,width,height,org_x1,org_y1,org_x2,org_y2,text,x,y,w,h
31429,m01-084.png,1,2479,3542,967,1823,972,1828,.,0.39,0.52,0.00,0.00
31430,m01-084.png,1,2479,3542,352,780,638,878,Deadly,0.20,0.23,0.12,0.03
31431,m01-084.png,1,2479,3542,712,772,1005,841,stillness,0.35,0.23,0.12,0.02
31432,m01-084.png,1,2479,3542,1018,825,1040,870,",",0.42,0.24,0.01,0.01
31433,m01-084.png,1,2479,3542,1092,775,1351,893,deadly,0.49,0.24,0.10,0.03
31434,m01-084.png,1,2479,3542,1432,785,1690,869,portent,0.63,0.23,0.10,0.02
31435,m01-084.png,1,2479,3542,1727,753,1743,832,!,0.70,0.22,0.01,0.02
31436,m01-084.png,1,2479,3542,363,966,566,1025,Steve,0.19,0.28,0.08,0.02
31437,m01-084.png,1,2479,3542,645,964,1046,1021,awakened,0.34,0.28,0.16,0.02
31438,m01-084.png,1,2479,3542,1135,961,1304,1057,early,0.49,0.28,0.07,0.03


In [22]:
"""
    gather all words
"""

import re,os
regex = re.compile('[^a-zA-Z]')
allWords=[]

#images=['c04-139.png', 'g06-018r.png', 'a01-072x.png', 'h01-010.png', 'c04-050.png', 'm01-032.png', 'g06-042r.png', 'g06-026i.png', 'h07-080.png', 'e04-022.png','g06-047h.png']

images=["m01-084.png"]#,"m01-084.png",'m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

orgImage=images[0]

#testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/"+orgImage

testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/"+orgImage

for indx,rows in temp.iterrows():
    
    word=rows["text"]
    word= regex.sub('', word)

    if len(word)>1:
        allWords.append(word)
    
allWords=list(set(allWords))
test_word_label=get_comb_label(allWords)
len(allWords)
#allWords

44

In [ ]:
import cv2
import os


testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"+orgImage+"//"
savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/final//"
image=cv2.imread(testImgPath+orgImage)
print(" original image shape:",image.shape)

matchCount=0
missMatch=0

for indx,imgName in enumerate(os.listdir(testCropPath)):
    
    print(" imgName:",imgName)
    word=imgName.split("]_")[1]
    word=word.split("_.png")[0]
    
    cor=imgName.split("[")[1]
    cor=cor.split("]")[0]
    cor="["+cor+"]"
    cor=eval(cor)
    
    x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
        
    #imgName=imgName.split("_")[0]
    
    
    predWord,score=predict(testCropPath+imgName)
    
    print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
    print("")
    
    if word==predWord:
        #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
        #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        matchCount+=1
    else:
        
        cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
        cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        missMatch+=1
        
cv2.imwrite(savePath+orgImage,image)

print("match",matchCount)
print("miss match:",missMatch)
print(" Accuracy:",matchCount/(matchCount+missMatch))


In [52]:
import os
os.getcwd()

'/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3'